In [ ]:
import os
import time
import warnings
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime

In [ ]:
# API KEY WeatherAPI = c2436f127ed34680b04182951232210

# Armado de la URL

In [ ]:
# variable que almacena la ciudad que queremos consultar
query = 'Santiago'

# La key que nos asigne Weather api
api_key = 'c2436f127ed34680b04182951232210'

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
print(url_clima)

http://api.weatherapi.com/v1/forecast.json?key=c2436f127ed34680b04182951232210&q=Santiago&days=1&aqi=no&alerts=no


In [ ]:
response = requests.get(url_clima).json()
response

{'location': {'name': 'Santiago',
  'region': 'Region Metropolitana',
  'country': 'Chile',
  'lat': -33.45,
  'lon': -70.67,
  'tz_id': 'America/Santiago',
  'localtime_epoch': 1698013761,
  'localtime': '2023-10-22 19:29'},
 'current': {'last_updated_epoch': 1698012900,
  'last_updated': '2023-10-22 19:15',
  'temp_c': 16.0,
  'temp_f': 60.8,
  'is_day': 1,
  'condition': {'text': 'Overcast',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png',
   'code': 1009},
  'wind_mph': 13.6,
  'wind_kph': 22.0,
  'wind_degree': 170,
  'wind_dir': 'S',
  'pressure_mb': 1016.0,
  'pressure_in': 30.0,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 72,
  'cloud': 100,
  'feelslike_c': 16.0,
  'feelslike_f': 60.8,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 4.0,
  'gust_mph': 6.2,
  'gust_kph': 9.9},
 'forecast': {'forecastday': [{'date': '2023-10-22',
    'date_epoch': 1697932800,
    'day': {'maxtemp_c': 22.5,
     'maxtemp_f': 72.6,
     'mintemp_c': 13.4,
     'mintemp_f': 56.1

In [ ]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [ ]:
# Usare solo Forecast para el entregable
# Obtenemos las keys para el pronostico del dia
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [ ]:
# compruebo que el largo de la 'hora' sea de 24, asi me aseguro que informara en intervalos de 1 hora
# de 00:00 hrs a 23:00 hrs
len(response['forecast']['forecastday'][0]['hour'])

24

In [ ]:
# FECHA
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0]

'2023-10-22'

In [ ]:
# HORA, como una variable entera
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

1

In [ ]:
# Condicion del clima
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

'Partly cloudy'

In [ ]:
# TEMPERATURA en grados Celsius
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

14.3

In [ ]:
# ¿LLovera o no llovera?
# 0 : No lloverá ; 1: Lloverá
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [ ]:
# Probabilidad de lluvia
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

0

# Estructuración Dataframe

In [ ]:
# Funcion que nos va a permitir extraer los campos de los 24 registros de cada hora de la api (weather api)

def get_forecast(response, i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha, hora, condicion, tempe, rain, prob_rain

In [ ]:
# lista que recibira los registros
datos = []
for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    datos.append(get_forecast(response, i))

100%|███████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 24047.61it/s]


In [ ]:
# Comprobamos los registros
datos

[('2023-10-22', 0, 'Partly cloudy', 14.3, 0, 0),
 ('2023-10-22', 1, 'Partly cloudy', 13.9, 0, 0),
 ('2023-10-22', 2, 'Partly cloudy', 14.0, 0, 0),
 ('2023-10-22', 3, 'Partly cloudy', 13.7, 0, 0),
 ('2023-10-22', 4, 'Partly cloudy', 13.4, 0, 0),
 ('2023-10-22', 5, 'Partly cloudy', 13.7, 0, 0),
 ('2023-10-22', 6, 'Partly cloudy', 13.4, 0, 0),
 ('2023-10-22', 7, 'Partly cloudy', 14.6, 0, 0),
 ('2023-10-22', 8, 'Partly cloudy', 15.6, 0, 0),
 ('2023-10-22', 9, 'Partly cloudy', 16.7, 0, 0),
 ('2023-10-22', 10, 'Partly cloudy', 17.7, 0, 0),
 ('2023-10-22', 11, 'Partly cloudy', 18.9, 0, 0),
 ('2023-10-22', 12, 'Partly cloudy', 20.2, 0, 0),
 ('2023-10-22', 13, 'Partly cloudy', 21.3, 0, 0),
 ('2023-10-22', 14, 'Partly cloudy', 21.8, 0, 0),
 ('2023-10-22', 15, 'Patchy rain possible', 22.1, 1, 72),
 ('2023-10-22', 16, 'Patchy rain possible', 21.3, 0, 60),
 ('2023-10-22', 17, 'Patchy rain possible', 20.2, 1, 87),
 ('2023-10-22', 18, 'Cloudy', 18.8, 0, 0),
 ('2023-10-22', 19, 'Overcast', 16.0, 0, 0)

In [ ]:
# Columnas del Dataframe
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','Prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,Prob_lluvia
0,2023-10-22,0,Partly cloudy,14.3,0,0
1,2023-10-22,1,Partly cloudy,13.9,0,0
2,2023-10-22,2,Partly cloudy,14.0,0,0
3,2023-10-22,3,Partly cloudy,13.7,0,0
4,2023-10-22,4,Partly cloudy,13.4,0,0
5,2023-10-22,5,Partly cloudy,13.7,0,0
6,2023-10-22,6,Partly cloudy,13.4,0,0
7,2023-10-22,7,Partly cloudy,14.6,0,0
8,2023-10-22,8,Partly cloudy,15.6,0,0
9,2023-10-22,9,Partly cloudy,16.7,0,0


In [ ]:
# Código SQL de la tabla
sql_code = """
CREATE TABLE santiago_clima (
   Fecha TIMESTAMP,
   Hora TIME,
   Condicion VARCHAR,
   Temperatura FLOAT,
   Lluvia FLOAT,
   Prob_lluvia FLOAT
)
DISTKEY (Fecha)
SORTKEY (Hora);
"""

# Nombre del archivo en el que se guarda el código SQL
nombre_archivo = "codigo_tabla.sql"

# Abre el archivo en modo escritura y guarda el código SQL
with open(nombre_archivo, "w") as archivo:
    archivo.write(sql_code)

print(f"El código SQL ha sido guardado en el archivo '{nombre_archivo}'.")


### Justificación Llaves de orden (SOTRKEY) y distribución (DISTKEY):

-  Si se desea almacenar información de una API por la columna 'Hora', es una buena idea utilizar 'Hora' como clave de ordenamiento, esto permitirá que las filas estén físicamente ordenadas por hora en Redshift, lo que facilitará la recuperación de datos en orden cronológico.

- La elección de la clave de distribución depende de cómo se planea realizar consultas en la tabla y de la naturaleza de los datos. En general, es aconsejable distribuir los datos de manera que las consultas comunes puedan ser ejecutadas de manera eficiente, dado que mi patron de consulta podria ser por fecha, se podrías usar 'Fecha' como clave de distribución. Por otro lado, si las consultas suelen involucrar múltiples columnas en igual medida, se podría considerar una distribución uniforme, en este caso conviene elegir una columna con alta cardinalidad o una columna de tipo hash para distribuir de manera uniforme los datos en los nodos de Redshift.

In [ ]:
'''
-- Ejemplo de la sentencia COPY para cargar datos desde un archivo CSV en Redshift
COPY tu_tabla
FROM 's3://ruta-de-tu-bucket/datos.csv'  -- Reemplaza con la ruta de tu archivo CSV en Amazon S3
CREDENTIALS 'aws_access_key_id=ACCESS_KEY_ID;aws_secret_access_key=SECRET_ACCESS_KEY'  -- Reemplaza con tus credenciales de AWS
CSV  -- Especifica el formato del archivo
IGNOREHEADER 1  -- Si el archivo CSV tiene una fila de encabezado, ignórala
DELIMITER ',';  -- Especifica el delimitador utilizado en el archivo CSV

-- Ejemplo:
-- COPY tu_tabla
-- FROM 's3://mi-bucket/datos.csv'
-- CREDENTIALS 'aws_access_key_id=MI_ACCESS_KEY;aws_secret_access_key=MI_SECRET_KEY'
-- CSV
-- IGNOREHEADER 1
-- DELIMITER ',';

'''

In [63]:
# Solo para codear un poco mas
# Eleccion de un horario adecuado: lluvia en horario vespertino
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)
df_rain

,Condicion
Hora,
15,Patchy rain possible
17,Patchy rain possible
20,Patchy rain possible
21,Patchy rain possible
